# Parse Web pages and extract content to CSV

In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import time
from selenium import webdriver
import os
import random
import pyprind
from pprint import pprint
import pandas as pd

In [2]:
container_node = soup.find(class_=['detail_view'])
details_node = container_node.find(class_=['individual_internship'])

NameError: name 'soup' is not defined

In [3]:
def get_domain_and_company(node):
    company_node = node.find(class_=['company'])
    domain_profile = company_node.select('.heading_4_5.profile')
    company_name = company_node.select('.heading_6.company_name')
    return {'domain_profile': domain_profile[0].text.strip(), 'company_name': company_name[0].text.strip()}

In [4]:
DATA = dict()

def store(key: str, value: str):
    if(value is not None):
        DATA[key.strip()] = value.strip()
    else:
        print('value is none')

In [5]:
def get_location(node):
    internship_meta_node=node.find(class_='individual_internship_details')
    location_link = internship_meta_node.select('.location_link')
    if(len(location_link) > 0):
        location = location_link[0].text
        return {'location': location}
    return None

In [6]:
def get_internship_metadata(node):
    details_container = node.find(class_='internship_other_details_container')
    details_node = node.find(class_=['individual_internship'])
    rows_node = node.select('.other_detail_item ')
    info = dict()
    for node in rows_node:
        head = node.find(class_='item_heading')
        detail = node.find(class_='item_body')
        store(head.text, detail.text)
        if(head != None and detail != None):
            info[head.text.strip()] = detail.text.strip()
    num_applicatants = details_node.find(class_=['applications_message'])
    if(num_applicatants):
        info['applicants'] = num_applicatants.text
    return info

In [16]:
def get_internship_details(node):
    internship_details_node = node.find(class_='internship_details')
    sections = internship_details_node.find_all('div')

    contents = dict()
    cur_key = ''
    for node in sections:
        attrs = node.attrs
        if('class' in attrs):
            if('section_heading' in attrs['class']):
                cur_key = node.text.strip()
                contents[cur_key] = []
            if('text-container' in attrs['class'] or 'round_tabs_container' in attrs['class']):
                contents[cur_key].append(node.text.strip())
    old_key = list(contents.keys())[0]
    contents['company_details'] = contents[old_key]
    del contents[old_key]
    for val in contents:
        if type(contents[val]) is list:
            contents[val] = ';'.join(contents[val])
    return contents

In [17]:
# ROOT_PATH = '../raw/internshala/internship-pages'
ROOT_PATH = '../raw/internshala-june/internship-pages'
all_pages = os.listdir(ROOT_PATH)

all_pages_data = []
bar = pyprind.ProgBar(len(all_pages))
for page_name in all_pages:
#     try:
    combined_path = ROOT_PATH + '/' + page_name
    data = dict()
    soup = BeautifulSoup(open(combined_path))
    container_node = soup.find(class_=['detail_view'])
    location = get_location(container_node)
    dnc = get_domain_and_company(container_node)
    details = get_internship_details(container_node)
    metadata = get_internship_metadata(container_node)
    data.update(location)
    data.update(dnc)
    data.update(details)
    data.update(metadata)
    all_pages_data.append(data)
# except Exception as e:
#         print('Exception = ' + str(e))
    bar.update()
# print(bar)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:06


In [18]:
s = set()
for x in all_pages_data:
    for k in x.keys():
        s.add(k)
        
s

{'',
 'About part time job/internship',
 'About the internship',
 'About the work from home job/internship',
 'Additional Information',
 'Apply By',
 'Duration',
 'Number of openings',
 'Other requirements',
 'Perks',
 'Skill(s) required',
 'Start Date',
 'Stipend',
 'Who can apply',
 'applicants',
 'company_details',
 'company_name',
 'domain_profile',
 'location'}

## Panda Time

+ load list of dictionary as pandas dataframe
+ light clean up of data
    + rename columns to something more meaningful
    + standardise column name casing
    + replace '\n' with ',' in skills
+ export to csv

In [19]:
df = pd.DataFrame(all_pages_data)

In [20]:
df.head(3)

,location,domain_profile,company_name,About the work from home job/internship,Skill(s) required,Who can apply,Perks,Number of openings,company_details,Start Date,Duration,Stipend,Apply By,applicants,Other requirements,About the internship,Additional Information,,About part time job/internship
0,Work From Home,Web Development,Perfect Skills,Selected intern's day-to-day responsibilities ...,PHP\nHTML\nCSS\nJavaScript\nPython\nWordPress,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,2,Website;Perfect Skills is committed to helping...,Starts immediatelyImmediately,4 Months,5000 /month,14 Jul' 21,46 applicants,NaN,NaN,NaN,NaN,NaN
1,Work From Home,Web Development,Geekster,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nAngularJS\nNode.js\nRea...,Only those candidates can apply who:\n1. are a...,Certificate\n Flexible work hours,1,Website;Geekster is on a mission to redefine t...,Starts immediatelyImmediately,3 Months,15000 /month,13 Jul' 21,53 applicants,1. Should have hands-on experience with Node.j...,NaN,NaN,NaN,NaN
2,Work From Home,Backend Development,Global Free School,Selected intern's day-to-day responsibilities ...,PHP\nMySQL\nHTML\nCSS\nJavaScript\njQuery\nCod...,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation,2,Website;We are a team of IIT graduates working...,Starts immediatelyImmediately,6 Months,10000 /month,4 Jul' 21,54 applicants,NaN,NaN,NaN,NaN,NaN


##### Replace '' with 'is_part_time'

In [23]:
df.columns # notice column name with name as '' (empty). Rename that to is_part_time

Index(['location', 'domain_profile', 'company_name',
       'About the work from home job/internship', 'Skill(s) required',
       'Who can apply', 'Perks', 'Number of openings', 'company_details',
       'Start Date', 'Duration', 'Stipend', 'Apply By', 'applicants',
       'Other requirements', 'About the internship', 'Additional Information',
       '', 'About part time job/internship'],
      dtype='object')

In [24]:
df.rename(columns={'':'is_part_time'}, inplace=True)

In [25]:
df.is_part_time = df.is_part_time.fillna(0) # replace NA with 0
df.is_part_time = df.is_part_time.replace('Part time allowed', 1) # otherwise 1

##### Fixing Columns Name Casings

In [26]:
# Logic for conversion: replace spaces with '_' and lowercase the entire string
columns_revision = [(i, i.replace(' ','_').lower()) for i in df.columns.to_list()]
old_n_new_columns = {conv[0]:conv[1] for conv in columns_revision}
pprint(columns_revision)
print('\n\ncolumns as dict\n\n')
pprint(old_n_new_columns)

[('location', 'location'),
 ('domain_profile', 'domain_profile'),
 ('company_name', 'company_name'),
 ('About the work from home job/internship',
  'about_the_work_from_home_job/internship'),
 ('Skill(s) required', 'skill(s)_required'),
 ('Who can apply', 'who_can_apply'),
 ('Perks', 'perks'),
 ('Number of openings', 'number_of_openings'),
 ('company_details', 'company_details'),
 ('Start Date', 'start_date'),
 ('Duration', 'duration'),
 ('Stipend', 'stipend'),
 ('Apply By', 'apply_by'),
 ('applicants', 'applicants'),
 ('Other requirements', 'other_requirements'),
 ('About the internship', 'about_the_internship'),
 ('Additional Information', 'additional_information'),
 ('is_part_time', 'is_part_time'),
 ('About part time job/internship', 'about_part_time_job/internship')]


columns as dict


{'About part time job/internship': 'about_part_time_job/internship',
 'About the internship': 'about_the_internship',
 'About the work from home job/internship': 'about_the_work_from_home_job/inter

In [27]:
# replace old name with new
df.rename(columns=old_n_new_columns, inplace=True)
df

,location,domain_profile,company_name,about_the_work_from_home_job/internship,skill(s)_required,who_can_apply,perks,number_of_openings,company_details,start_date,duration,stipend,apply_by,applicants,other_requirements,about_the_internship,additional_information,is_part_time,about_part_time_job/internship
0,Work From Home,Web Development,Perfect Skills,Selected intern's day-to-day responsibilities ...,PHP\nHTML\nCSS\nJavaScript\nPython\nWordPress,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,2,Website;Perfect Skills is committed to helping...,Starts immediatelyImmediately,4 Months,5000 /month,14 Jul' 21,46 applicants,NaN,NaN,NaN,0,NaN
1,Work From Home,Web Development,Geekster,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nAngularJS\nNode.js\nRea...,Only those candidates can apply who:\n1. are a...,Certificate\n Flexible work hours,1,Website;Geekster is on a mission to redefine t...,Starts immediatelyImmediately,3 Months,15000 /month,13 Jul' 21,53 applicants,1. Should have hands-on experience with Node.j...,NaN,NaN,0,NaN
2,Work From Home,Backend Development,Global Free School,Selected intern's day-to-day responsibilities ...,PHP\nMySQL\nHTML\nCSS\nJavaScript\njQuery\nCod...,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation,2,Website;We are a team of IIT graduates working...,Starts immediatelyImmediately,6 Months,10000 /month,4 Jul' 21,54 applicants,NaN,NaN,NaN,0,NaN
3,Mumbai,Web Development,Knitroot India Private Limited,NaN,WordPress,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,1,Website;We specialize in occasion-based mercha...,Starts immediatelyImmediately,1 Month,5000 /month,13 Jul' 21,38 applicants,1. Candidates who have experience of 1 year in...,Selected intern's day-to-day responsibilities ...,NaN,0,NaN
4,Work From Home,Web Development,FitnessForce,Selected intern's day-to-day responsibilities ...,Python\nNode.js\nReactJS,Only those candidates can apply who:\n1. are a...,Letter of recommendation\n Flexible work hours...,2,Website;FitnessForce is a SaaS-based CRM club ...,Starts immediatelyImmediately,6 Months,10000 /month,5 Jul' 21,66 applicants,1. Looking for a fresher computer programmer i...,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Mumbai,Odoo Development,Best Roadways Limited,NaN,Odoo,Only those candidates can apply who:\n1. are a...,Certificate,1,Website;Best Roadways was founded in 1985 with...,Starts immediatelyImmediately,3 Months,10000 /month,2 Jul' 21,Be an early applicant,1. Must have strong knowledge of Odoo,Selected intern's day-to-day responsibilities ...,NaN,0,NaN
1209,Work From Home,Web Development,Dot2Globe Solutions India Private Limited,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nUI & UX Design\nFigma,Only those candidates can apply who:\n1. are a...,Certificate\n 5 days a week,1,Website;Dot2Globe Solutions is a start-up orga...,Starts immediatelyImmediately,3 Months,5000-8000 /month,14 Jul' 21,29 applicants,1. Must have worked as UI/UX design intern (wi...,NaN,Note: Extension of internship/offer of full-ti...,0,NaN
1210,Work From Home,Web Development,TikiLIVE,Selected intern's day-to-day responsibilities ...,NaN,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation,3,Website;We make OTT delivery solutions worldwi...,Starts immediatelyImmediately,1 Month,20000 /month,2 Jul' 21,391 applicants,NaN,NaN,NaN,1,NaN
1211,Work From Home,Web Development,Success Genix,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nREST API\nReactJS\nAmazon Web Serve...,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,2,Success Genix is a team of people that make ou...,Starts immediatelyImmediately,2 Months,3000 /month,6 Jul' 21,100 applicants,NaN,NaN,NaN,1,NaN


#### Convert DF to CSV and store it locally

In [28]:
df.to_csv('internshala-cs-internships-june.csv')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   location                                 1213 non-null   object
 1   domain_profile                           1213 non-null   object
 2   company_name                             1213 non-null   object
 3   about_the_work_from_home_job/internship  818 non-null    object
 4   skill(s)_required                        1070 non-null   object
 5   who_can_apply                            1213 non-null   object
 6   perks                                    1154 non-null   object
 7   number_of_openings                       1213 non-null   object
 8   company_details                          1213 non-null   object
 9   start_date                               1213 non-null   object
 10  duration                                 1213 non-null   obj

In [280]:
df.is_part_time

0       1
1       0
2       0
3       0
4       0
       ..
2369    1
2370    0
2371    0
2372    0
2373    0
Name: is_part_time, Length: 2374, dtype: int64

In [300]:
pprint(all_pages_data[100])

{'About the internship': "Selected intern's day-to-day responsibilities "
                         'include:\n'
                         '\n'
                         '1. Ensuring quality of print & digital content by '
                         'conducting pre-release testing\n'
                         '2. Reporting and documenting errors & technical '
                         'issues\n'
                         '3. Transcribing curriculum content/data into the '
                         'required electronic format\n'
                         '4. Assisting content development with image search & '
                         'fact-checking',
 'Additional Information': 'We are looking for a quality assurance intern who '
                           'is excited by the idea of democratizing quality '
                           'teaching tools and techniques to students and '
                           'teachers in APS.',
 'Apply By': "11 Jun' 21",
 'Duration': '3 Months',
 'Number of opening